In [30]:
# An example of building up the structure of a figure as a Python dictionary
# and then constructing a graph object figure from that dictionary.

In [31]:
# https://plotly.com/python/animations/#using-a-slider-and-buttons
# https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv

In [46]:
import pandas as pd

In [47]:
dataset = pd.read_csv('output_data/time_series/rolling-time-series.csv')

In [48]:
#dataset['Date_Confirmed'] = pd.to_datetime(dataset.Date_Confirmed, format='%d-%m-%Y', errors='coerce')
#dataset['Date_Confirmed'] = dataset['Date_Confirmed'].dt.strftime('%d-%m-%Y')

In [49]:
from datetime import datetime

dataset['Date_Confirmed'] = dataset['Date_Confirmed'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [50]:
# https://community.plotly.com/t/plotly-doesnt-load-most-of-the-time/32095/3

import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'chrome'

In [51]:
# Make list of regions

regions = []

for region in dataset['Region']:
    if region not in regions:
        regions.append(region)

In [52]:
years = sorted(list(set(dataset['Date_Confirmed'])))

In [54]:
filled_df = []
for region in regions:
    for year in years:
        dataset_by_year = dataset[dataset['Date_Confirmed'] == year]
        dataset_by_year_and_region = dataset_by_year[dataset_by_year['Region'] == region]
        if len(dataset_by_year_and_region)==0:
            dataset_by_year_less = dataset[dataset['Date_Confirmed'] < year]
            dataset_by_year_and_region_less = dataset_by_year_less[dataset_by_year_less['Region'] == region]
            if len(dataset_by_year_and_region_less)==0:
                filled_df.append({'Date_Confirmed': year, 'Region': region, 'total':0, 'daily':0})
            else:
                last_dict = dataset_by_year_and_region_less.iloc[0].to_dict()
                last_dict['Date_Confirmed'] = year
                filled_df.append(last_dict)
        else:
            filled_df.append(dataset_by_year_and_region.iloc[0].to_dict())

In [55]:
dataset = pd.DataFrame(filled_df)

In [56]:
dataset['total'].max()

729006.0

In [57]:
dataset['daily'].max()

31564.0

In [58]:
dataset.head()

,Date_Confirmed,Region,total,daily
0,2020-03-05,China,2422.0,8.0
1,2020-03-06,China,2430.0,8.0
2,2020-03-07,China,2436.0,6.0
3,2020-03-08,China,2440.0,4.0
4,2020-03-09,China,2443.0,3.0


In [59]:
#dataset['Date_Confirmed'] = pd.to_datetime(dataset.Date_Confirmed, format='%Y-%m-%d', errors='coerce')

In [60]:
#dataset.to_csv('test.csv', index=False)

In [61]:
len(str(dataset['total'].max()))+1

9

In [62]:
# Make figure

fig_dict = {
    'data': [],
    'layout': {},
    'frames': []
}

In [63]:
# Fill in most of layout

fig_dict['layout']['xaxis'] = {'title': 'casos totales', 'type': 'log', 'range': [0,6]}
fig_dict['layout']['yaxis'] = {'title': 'casos diarios', 'type': 'log', 'range': [0,6]}
fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': datetime.strptime('15-02-2020', '%d-%m-%Y').isoformat().split('T')[0],
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
    }
fig_dict['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                                'fromcurrent': True, 'transition': {'duration': 300,
                                                                    'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False},
                                  'mode': 'immediate',
                                  'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Fecha:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [64]:
year = datetime.strptime('2020-03-05', '%Y-%m-%d')

for region in regions:
    
    dataset_by_year = dataset[dataset['Date_Confirmed'] == year]
    dataset_by_year_and_region = dataset_by_year[dataset_by_year['Region'] == region]
    
    data_dict = {
        'x': list(dataset_by_year_and_region['total']),
        'y': list(dataset_by_year_and_region['daily']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_region['Region']),
    }
    fig_dict['data'].append(data_dict)

In [66]:
# Make frames

for year in years:
    
    frame = {'data': [], 'name': str(year.isoformat().split('T')[0])}
    dataset_by_year = dataset[dataset['Date_Confirmed'] <= year]
    for region in regions:
        dataset_by_year_and_region = dataset_by_year[
            dataset_by_year['Region'] == region]
        
        data_dict = {
                    'x': list(dataset_by_year_and_region['total']),
                    'y': list(dataset_by_year_and_region['daily']),
                    'mode': 'lines',
                    'text': list(dataset_by_year_and_region['Region']),
                    'name': region
            }
        frame['data'].append(data_dict)
        
    fig_dict['frames'].append(frame)
    slider_step = {'args': [
        [year.isoformat().split('T')[0]],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
         'transition': {'duration': 300}}
    ],
        'label': year.isoformat().split('T')[0],
        'method': 'animate'}
    sliders_dict['steps'].append(slider_step)
    
fig_dict['layout']['sliders'] = [sliders_dict]

In [67]:
#fig_dict['frames'][-1]

In [68]:
fig = go.Figure(fig_dict)

fig.show()